In [ ]:
# LangChainを使っていなくてもトレースは可能
import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable

client = wrap_openai(openai.Client())


@traceable
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")

In [ ]:
from langsmith import Client
from langsmith.evaluation import evaluate


client = Client()

dataset_name = "Sample Dataset"
dataset = client.create_dataset(dataset_name, description="A sample dataset in Langsmith.")
client.create_examples(
    inputs=[
        {"postfix": "to LangSmith"},
        {"postfix": "to Evaluations in LangSmith"},
    ],
    outputs = [
        {"output": "Welcome to LangSmith"},
        {"output": "Welcome to Evaluations in LangSmith"},
    ],
    dataset_id=dataset.id
)

def exact_match(run, example):
    return {"score": run.outputs["output"] == example.outputs["output"]}

experiment_results = evaluate(
    lambda input: "Welcome " + input["postfix"],
    data=dataset_name,
    evaluators=[exact_match],
    experiment_prefix="sample-experiment",
    metadata={
        "version": "1.0.0",
        "revision_id": "beta"
    },
)

In [ ]:
experiment_results